<a href="https://colab.research.google.com/github/akankshanehete/RNNtesting/blob/main/Smartphone_sensor_classification_newmethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization,Masking
from scipy import stats

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [75]:
!gdown --id 152sWECukjvLerrVG2NUO8gtMFg83RKCF --output WISDM_ar_latest.tar.gz
!tar -xvf WISDM_ar_latest.tar.gz

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=152sWECukjvLerrVG2NUO8gtMFg83RKCF
To: /content/WISDM_ar_latest.tar.gz
100% 11.4M/11.4M [00:00<00:00, 290MB/s]
WISDM_ar_v1.1/
WISDM_ar_v1.1/readme.txt
WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt
WISDM_ar_v1.1/WISDM_ar_v1.1_raw_about.txt
WISDM_ar_v1.1/WISDM_ar_v1.1_transformed.arff
WISDM_ar_v1.1/WISDM_ar_v1.1_trans_about.txt


In [76]:
column_names = ['user_id', 'activity', 'timestamp', 'x_axis', 'y_axis', 'z_axis']

df = pd.read_csv('WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt', header=None, names=column_names)
df.z_axis.replace(regex=True, inplace=True, to_replace=r';', value=r'')
df['z_axis'] = df.z_axis.astype(np.float64)
df.dropna(axis=0, how='any', inplace=True)
print(df.shape)
df = df[df['timestamp'] != 0]
print(df.shape)
df = df.drop(columns=['timestamp'])

(1098203, 6)
(1085360, 6)


In [77]:
df.head()

,user_id,activity,x_axis,y_axis,z_axis
0,33,Jogging,-0.694638,12.680544,0.503953
1,33,Jogging,5.012288,11.264028,0.953424
2,33,Jogging,4.903325,10.882658,-0.081722
3,33,Jogging,-0.612916,18.496431,3.023717
4,33,Jogging,-1.184970,12.108489,7.205164


In [78]:
users = list(df.groupby(['user_id']))
smartphonedfs = []
for i in range(len(users)):
  users[i] = users[i][1]

for i in range (len(users)):
  activ = list(users[i].groupby(['activity']))
  for i in range(len(activ)):
    smartphonedfs.append(activ[i][1])

print(smartphonedfs[0])
print(smartphonedfs[1])


        user_id    activity  x_axis  y_axis  z_axis
966741        1  Downstairs   -0.15    9.15   -0.34
966742        1  Downstairs    0.11    9.19    2.76
966743        1  Downstairs   -4.06    7.40    4.02
966744        1  Downstairs   -2.87    7.93    3.21
966745        1  Downstairs   -0.19   10.04    4.82
...         ...         ...     ...     ...     ...
971933        1  Downstairs   -0.61    9.72    2.07
971934        1  Downstairs    5.71   11.35    0.69
971935        1  Downstairs    8.62    5.41   -1.18
971936        1  Downstairs    4.25    7.59    0.08
971937        1  Downstairs   -0.34   11.30    0.72

[2941 rows x 5 columns]
        user_id activity  x_axis  y_axis  z_axis
948761        1  Jogging    4.48   14.18   -2.11
948762        1  Jogging    3.95   12.26   -2.68
948763        1  Jogging    6.05    9.72   -1.95
948764        1  Jogging    5.24    7.21   -5.56
948765        1  Jogging    7.27    5.79   -6.51
...         ...      ...     ...     ...     ...
965872  

In [79]:
columns = ['x_axis', 'y_axis', 'z_axis']
x_columns = ['x_axis']
y_columns = ['y_axis']
z_columns = ['z_axis']


In [80]:
a =[]
actiontargets = []
subjectids = []
smartphonedflens = []

actions = ['Downstairs','Jogging','Upstairs','Walking', 'Sitting','Standing']
num_classes = len(actions)
window_size = 15
testing_percent = 0.5

subjectids = []
for sensordf in smartphonedfs:
  actiontargets.append(actions.index(list(sensordf['activity'])[0]))
  subjectids.append(list(sensordf['user_id'])[0])
  smartphonedflens.append(len(sensordf))

print(len(smartphonedfs))
print(actiontargets)
print(subjectids)
print(smartphonedflens)
pd.Series(smartphonedflens).describe()

179
[0, 1, 2, 3, 1, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 3, 0, 1, 5, 2, 3, 0, 1, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 4, 5, 2, 3, 0, 1, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 4, 5, 2, 3, 1, 3, 0, 1, 2, 3, 0, 1, 4, 5, 2, 3, 0, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 1, 4, 5, 3, 0, 1, 4, 5, 2, 3]
[1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 25, 25, 26, 26, 26, 26, 27,

count      179.000000
mean      6063.463687
std       4634.205648
min        544.000000
25%       2341.500000
50%       3892.000000
75%      10989.500000
max      17622.000000
dtype: float64

In [81]:
def plot_activity(smartphonedf):
  plt.plot(smartphonedf[columns])
  plt.title('Subject '+ str(list(smartphonedf['user_id'])[0]) + " " + str(list(smartphonedf['activity'])[0]))
  plt.show()



In [82]:
for i in range (len (smartphonedfs)):
  smartphonedfs[i] = smartphonedfs[i].reset_index()
  smartphonedfs[i] = smartphonedfs[i].drop(columns=['index'])

print(smartphonedfs[0])

      user_id    activity  x_axis  y_axis  z_axis
0           1  Downstairs   -0.15    9.15   -0.34
1           1  Downstairs    0.11    9.19    2.76
2           1  Downstairs   -4.06    7.40    4.02
3           1  Downstairs   -2.87    7.93    3.21
4           1  Downstairs   -0.19   10.04    4.82
...       ...         ...     ...     ...     ...
2936        1  Downstairs   -0.61    9.72    2.07
2937        1  Downstairs    5.71   11.35    0.69
2938        1  Downstairs    8.62    5.41   -1.18
2939        1  Downstairs    4.25    7.59    0.08
2940        1  Downstairs   -0.34   11.30    0.72

[2941 rows x 5 columns]


In [83]:
# for smartphonedf in smartphonedfs:
#   plot_activity(smartphonedf)
  
  

In [84]:
a =[]
actiontargets = []
subjectids = []
smartphonedflens = []

actions = ['Downstairs','Jogging','Upstairs','Walking', 'Sitting','Standing']
num_classes = len(actions)
window_size = 50
testing_percent = 0.5

subjectids = []
for sensordf in smartphonedfs:
  actiontargets.append(actions.index(list(sensordf['activity'])[0]))
  subjectids.append(list(sensordf['user_id'])[0])
  smartphonedflens.append(len(sensordf))

print(len(smartphonedfs))
print(actiontargets)
print(subjectids)
print(smartphonedflens)
pd.Series(smartphonedflens).describe()

179
[0, 1, 2, 3, 1, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 3, 0, 1, 5, 2, 3, 0, 1, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 4, 5, 2, 3, 0, 1, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 4, 5, 2, 3, 1, 3, 0, 1, 2, 3, 0, 1, 4, 5, 2, 3, 0, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 0, 1, 4, 5, 2, 3, 1, 4, 5, 3, 0, 1, 4, 5, 2, 3]
[1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 25, 25, 26, 26, 26, 26, 27,

count      179.000000
mean      6063.463687
std       4634.205648
min        544.000000
25%       2341.500000
50%       3892.000000
75%      10989.500000
max      17622.000000
dtype: float64

In [85]:
# creating gaussian windowing dataframes

gaussian_velo = ['x_axis_avg','x_axis_std', 'y_axis_avg','y_axis_std', 'z_axis_avg','z_axis_std']

def extract_gaussian_windowing(sensordf, period): 
  windowdf = pd.DataFrame(columns=gaussian_velo)
  for column in columns: 
    # using the mean and variance for the windowing approach
    windowdf[column+'_avg'] = sensordf[column].groupby(np.arange(len(sensordf))//period).mean()
    windowdf[column+'_std'] = sensordf[column].groupby(np.arange(len(sensordf))//period).var()
  return windowdf.dropna() 



In [86]:
gaussiandf = extract_gaussian_windowing(smartphonedfs[0],100)
gaussiandf.head(10)

gaussiandfs = []
gaussiandflens = []

for sensordf in smartphonedfs: 
  gaussiandf = extract_gaussian_windowing(sensordf, window_size)
  gaussiandfs.append(gaussiandf)
  gaussiandflens.append(len(gaussiandf))

pd.Series(gaussiandflens).describe()

count    179.000000
mean     121.748603
std       92.689759
min       11.000000
25%       47.500000
50%       78.000000
75%      220.500000
max      353.000000
dtype: float64

In [87]:
# creating sequences from the windowing dataframes to feed into the RNN
sequences = list()


for i in range(len(gaussiandfs)):
  values = gaussiandfs[i][gaussian_velo].values
  sequences.append(values)

print(len(sequences))
print(len(actiontargets))

179
179


In [88]:
# padding each sequence with a certain value in order to make them all the same length because RNN does not accept variable sequence lengths
to_pad = 3000
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    # -1 is the special value that sequences are padded with that the RNN can mask
    last_val = -100
    n = to_pad - len_one_seq
   
    to_concat = np.full((n, len(one_seq[0])), -100)
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)


# truncate each sequence to the desired length (e.g how many windows in each training sequence?)
from keras.preprocessing.sequence import pad_sequences
seq_len = 177
final_seq = pad_sequences(new_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

sequences = np.array(final_seq)
sequences.shape

(179, 177, 6)

In [89]:
#splitting into training and testing sets 

# how many samples for testing and how many for training? in this case
testingnum = int(len(smartphonedfs)*testing_percent)

x_train = np.array(sequences[1:-testingnum])
y_train = np.array(actiontargets[1:-testingnum])

x_test = np.array(sequences[-testingnum:])
y_test = np.array(actiontargets[-testingnum:])

In [90]:
# model constants
BATCH_SIZE = 50
EPOCHS = 10

In [91]:
# building the rnn model (adding layers)

accuracies = []
for _ in range (10):
  model = Sequential()
  # add masking layer to the RNN to avoid the padded values in series since they are of variable length
  model.add(tf.keras.layers.Masking(mask_value=-100, input_shape=(seq_len, 6)))
  model.add(LSTM(128, input_shape=(x_train.shape[1:]), return_sequences=True))

  model.add(Dropout(0.2))
  model.add(BatchNormalization())  

  model.add(LSTM(128, return_sequences=True))
  model.add(Dropout(0.1))
  model.add(BatchNormalization())

  model.add(LSTM(128))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.2))

  # last layer has 2 nodes due to there being binary classification
  model.add(Dense(6, activation='softmax'))

  opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
    
  model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'])


  # training the model
  model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_test,y_test), shuffle=False)
  result = model.evaluate(x_test,y_test)
  accuracies.append(result[1])

mean_accuracy = sum(accuracies)/len(accuracies)
print('mean accuracy: ' + str(mean_accuracy))
print('standard_deviation:' + str(np.std(accuracies)))

Epoch 1/10
2/2 [==============================] - 20s 6s/step - loss: 1.9195 - accuracy: 0.3596 - val_loss: 1.6614 - val_accuracy: 0.4045
Epoch 2/10
2/2 [==============================] - 3s 1s/step - loss: 1.2309 - accuracy: 0.5393 - val_loss: 1.5810 - val_accuracy: 0.4270
Epoch 3/10
2/2 [==============================] - 3s 2s/step - loss: 0.9641 - accuracy: 0.6180 - val_loss: 1.5264 - val_accuracy: 0.4270
Epoch 4/10
2/2 [==============================] - 3s 2s/step - loss: 0.7275 - accuracy: 0.6854 - val_loss: 1.4912 - val_accuracy: 0.4382
Epoch 5/10
2/2 [==============================] - 2s 1s/step - loss: 0.7037 - accuracy: 0.7528 - val_loss: 1.4620 - val_accuracy: 0.4382
Epoch 6/10
2/2 [==============================] - 2s 1s/step - loss: 0.6264 - accuracy: 0.7640 - val_loss: 1.4367 - val_accuracy: 0.4382
Epoch 7/10
2/2 [==============================] - 2s 1s/step - loss: 0.5840 - accuracy: 0.7640 - val_loss: 1.4157 - val_accuracy: 0.4494
Epoch 8/10
2/2 [========================